In [1]:
import numpy as np
from BPnumba.ABCnum import DABC,createDABC
from BPnumba.GeneticOperators import InstancePob,CreatePoblation,CreateHeuristicPob
import matplotlib.pyplot as plt
from BPmodule.BPGen  import PROBLEM2,PROBLEM3,PROBLEM4,PROBLEM5
from numba.typed import List as NumbaList
import random as rd
import numba as nb

In [2]:
P2A1 = PROBLEM2[0]
P2A2 = PROBLEM2[1]
P3A1 = PROBLEM3[0]
P3A2 = PROBLEM3[1]
P4A1 = PROBLEM4[0]
P4A2 = PROBLEM4[1]
P5A1 = PROBLEM5[0]
P5A2 = PROBLEM5[1]

In [3]:
contenedor =P3A1[0][0]
datos =P3A1[1]
idBoxes = np.arange(1,len(datos)+1,dtype=np.int64)
n=len(datos)
numItr = 1000
num_pop= 50
poblation = NumbaList(CreateHeuristicPob(num_pop, datos))
n

50

In [8]:
abc = createDABC(num_pop,n)

In [9]:
ColonyWorker = InstancePob(poblation,datos,contenedor)

In [ ]:
abc.Train(numItr,ColonyWorker,datos,contenedor)

In [ ]:
abc.BestInd.fi,n-abc.BestInd.load

In [ ]:
abc.BestInd.load

In [ ]:
abc.BestInd.codeSolution

In [ ]:
plt.figure()
plt.plot(abc.bestfi)
plt.ylim(0,1)
plt.show()